# Part One:
Move Step Seven of the risk_planning_progress_v2 notebook to a new notebook and import the risk project modules. Complete the stepwise elicitation step in the notebook and test it with different utility parameters. 

I completed this part of the project in two formats:

1. By creating an *input* stepwise algorithm with user input in the console. 

2. By creating an *automatic* stepwise algorithm which uses the utility functions provided.

## Input Stepwise Algorithim

In [3]:
import helpers as hlpr
import lottery as lot
import utility_functions as uf
import agent
import holt_laury as hl
import random
import matplotlib.pyplot as plt

def input_stepwise_elict(money_min = 0, money_max = 100):

    xp = [money_min,money_max]
    fp = [money_min,money_max]
    lottery_ce1 = [{'out': money_min, 'prob': 0.5}, {'out': money_max, 'prob': 0.5}]
    print(f"\nThe lottery has a 50/50 probability that it will either pay out {lottery_ce1[0].get('out')} or {lottery_ce1[1].get('out')}")

    print("\nIf I offered you a guaranteed amount of money instead of the chance to win this lottery,\nhow much would you need for you to be just as happy with the guaranteed money as you \nare with the lottery?")
    ce_1 = float(input())
    ev_1 = agent.expected_value(lottery_ce1)
    xp.append(ev_1)
    fp.append(ce_1)
    print(f"Your Answer: {ce_1}")

    lottery_ce2 = [{'out': money_min, 'prob': 0.5}, {'out': ce_1, 'prob': 0.5}]
    print(f"\nThe lottery has a 50/50 probability that it will either pay out {lottery_ce2[0].get('out')} or {lottery_ce2[1].get('out')}")
    
    print("\nIf I offered you a guaranteed amount of money instead of the chance to win this lottery,\nhow much would you need for you to be just as happy with the guaranteed money as you \nare with the lottery?")
    ce_2 = float(input())
    ev_2 = agent.expected_value(lottery_ce2)
    xp.append(ev_2)
    fp.append(ce_2)
    print(f"Your Answer: {ce_2}")

    lottery_ce3 = [{'out': ce_1, 'prob': 0.5}, {'out': money_max, 'prob': 0.5}]
    print(f"\nThe lottery has a 50/50 probability that it will either pay out {lottery_ce3[0].get('out')} or {lottery_ce3[1].get('out')}")
    
    print("\nIf I offered you a guaranteed amount of money instead of the chance to win this lottery,\nhow much would you need for you to be just as happy with the guaranteed money as you \nare with the lottery?")
    ce_3 = float(input())
    ev_3 = agent.expected_value(lottery_ce3)
    xp.append(ev_3)
    fp.append(ce_3)
    print(f"Your Answer: {ce_3}")

    events_n = 0
    print("\nHow many events would you like to run?")
    events_n = int(input())
    if events_n < 0:
        print()
        events_n = int(input("Negative numbers not accepted!\n How many events would you like to run?: "))
    else:
        for i in range(events_n):
            RanMin = random.randrange(money_min,money_max)
            RanMax = random.randrange(money_min, money_max)
            while RanMax < RanMin:
                RanMax = random.randrange(money_min, money_max)
            lottery_cen = [{'out': RanMin, 'prob': 0.5}, {'out': RanMax, 'prob': 0.5}]
            print(lottery_cen[0])
            print(f"\nThe lottery has a 50/50 probability that it will either pay out {lottery_cen[0].get('out')} or {lottery_cen[1].get('out')}")
            
            print("\nIf I offered you a guaranteed amount of money instead of the chance to win this lottery,\nhow much would you need for you to be just as happy with the guaranteed money as you \nare with the lottery?")
            ce_n = float(input())
            ev_n = agent.expected_value(lottery_cen)
            xp.append(ev_n)
            fp.append(ce_n)
            print(f"Your Answer: {ce_n}")

    xp.sort()
    fp.sort()

    slope_list = [(fp[i+1] - fp[i]) / (xp[i+1] - xp[i]) for i in range(len(xp) - 1)]
    intercept_list = [(slope_list[i]*(xp[0] - xp[i]) + fp[i]) for i in range(len(slope_list))]

    piecewise_list = [[{'slope': slope_list[x], 'intercept': intercept_list[x], 'x_min': xp[x], 'x_max': xp[x+1]}] for x in range(len(xp) - 1)]
    
    print("\nThe Piecewise Functions (intercept, slope, conditions):")
    print('---------------------------')
    print(f"{piecewise_list[0][0].get('intercept')}, {piecewise_list[0][0].get('slope')}x, if {piecewise_list[0][0].get('x_min')} ≤ x ≤ {piecewise_list[0][0].get('x_max')}")
    for subdomain in range(len(piecewise_list) - 1):
        print(f"{piecewise_list[subdomain+1][0].get('intercept')}, {piecewise_list[subdomain+1][0].get('slope')}x, if {piecewise_list[subdomain+1][0].get('x_min')} < x ≤ {piecewise_list[subdomain+1][0].get('x_max')}")

    return piecewise_list

In [4]:
input_stepwise_elict(money_min=0, money_max=100)


The lottery has a 50/50 probability that it will either pay out 0 or 100

If I offered you a guaranteed amount of money instead of the chance to win this lottery,
how much would you need for you to be just as happy with the guaranteed money as you 
are with the lottery?


ValueError: could not convert string to float: ''

## Auto Stepwise Algorithim

In [5]:
def auto_stepwise_elict(money_min = 0, money_max = 100, utilfn = uf.linear_utility, events_n = 0):
    
    xp = [money_min,money_max]
    fp = [money_min,money_max]

    lottery_ce1 = [{'out': money_min, 'prob': 0.5}, {'out': money_max, 'prob': 0.5}]
    print(f"\nThe lottery has a 50/50 probability that it will either pay out {lottery_ce1[0].get('out')} or {lottery_ce1[1].get('out')}")

    ce_1 = agent.expected_utility(lottery_ce1, utilfn)
    ev_1 = agent.expected_value(lottery_ce1)
    xp.append(ev_1)
    fp.append(ce_1)
    print(f"Your Answer: {ce_1}")

    lottery_ce2 = [{'out': money_min, 'prob': 0.5}, {'out': ce_1, 'prob': 0.5}]
    print(f"\nThe lottery has a 50/50 probability that it will either pay out {lottery_ce2[0].get('out')} or {lottery_ce2[1].get('out')}")
    ce_2 = agent.expected_utility(lottery_ce2, utilfn)
    ev_2 = agent.expected_value(lottery_ce2)
    xp.append(ev_2)
    fp.append(ce_2)
    print(f"Your Answer: {ce_2}")

    lottery_ce3 = [{'out': ce_1, 'prob': 0.5}, {'out': money_max, 'prob': 0.5}]
    print(f"\nThe lottery has a 50/50 probability that it will either pay out {lottery_ce3[0].get('out')} or {lottery_ce3[1].get('out')}")
    ce_3 = agent.expected_utility(lottery_ce3, utilfn)
    ev_3 = agent.expected_value(lottery_ce3)
    xp.append(ev_3)
    fp.append(ce_3)
    print(f"Your Answer: {ce_3}")

    if int(events_n) < 0:
        print()
        print("Negative numbers not accepted!")
    else:
        while len(xp) -5 < events_n:
            RanMin = random.randrange(money_min,money_max)
            RanMax = random.randrange(money_min, money_max)
            while RanMax < RanMin:
                RanMax = random.randrange(money_min, money_max)
            lottery_cen = [{'out': RanMin, 'prob': 0.5}, {'out': RanMax, 'prob': 0.5}]
            print(f"\nThe lottery has a 50/50 probability that it will either pay out {lottery_cen[0].get('out')} or {lottery_cen[1].get('out')}")
            ce_n = agent.certainty_equivalent(lottery_cen, utilfn, money_min, money_max, 0.01)[0]
            eu_n = agent.expected_utility(lottery_cen, utilfn)
            if ce_n not in xp:
                xp.append(ce_n)
                fp.append(eu_n)
                print(f"Your Answer: {ce_n}")

    xp.sort()
    fp.sort()

    slope_list = [(fp[i+1] - fp[i]) / (xp[i+1] - xp[i]) for i in range(len(xp) - 1)]
    intercept_list = [(slope_list[i]*(xp[0] - xp[i]) + fp[i]) for i in range(len(slope_list))]

    piecewise_list = [[{'slope': slope_list[x], 'intercept': intercept_list[x], 'x_min': xp[x], 'x_max': xp[x+1]}] for x in range(len(xp) - 1)]
    
    print("\nThe Piecewise Functions (intercept, slope, conditions):")
    print('---------------------------')
    print(f"{piecewise_list[0][0].get('intercept')}, {piecewise_list[0][0].get('slope')}x, if {piecewise_list[0][0].get('x_min')} ≤ x ≤ {piecewise_list[0][0].get('x_max')}")
    for subdomain in range(len(piecewise_list) - 1):
        print(f"{piecewise_list[subdomain+1][0].get('intercept')}, {piecewise_list[subdomain+1][0].get('slope')}x, if {piecewise_list[subdomain+1][0].get('x_min')} < x ≤ {piecewise_list[subdomain+1][0].get('x_max')}")

    return piecewise_list

In [7]:
print("LINEAR UTILITY")
auto_stepwise_elict(money_min = 0, money_max = 100, utilfn = uf.linear_utility, events_n = 5)
print("#   ------------------------------------------------------  #")

print("CONSTANT ABSOLUTE RISK AVERSION UTILITY")
auto_stepwise_elict(money_min = 0, money_max = 100, utilfn = uf.cara_utility, events_n = 5)
print("#   ------------------------------------------------------  #")

print("CONSTANT RELATIVE RISK AVERSION UTILITY")
auto_stepwise_elict(money_min = 0, money_max = 100, utilfn = uf.crra_utility, events_n = 5)
print("#   ------------------------------------------------------  #")

print("HYPERBOLIC ABOSOLUTE RISK AVERSION UTILITY")
auto_stepwise_elict(money_min = 0, money_max = 100, utilfn = uf.hara_utility, events_n = 5)
print("#   ------------------------------------------------------  #")

print("EXPONENTIAL UTILITY")
auto_stepwise_elict(money_min = 0, money_max = 100, utilfn = uf.exponential_utility, events_n = 5)
print("#   ------------------------------------------------------  #")

LINEAR UTILITY

The lottery has a 50/50 probability that it will either pay out 0 or 100
Your Answer: 50.0

The lottery has a 50/50 probability that it will either pay out 0 or 50.0
Your Answer: 25.0

The lottery has a 50/50 probability that it will either pay out 50.0 or 100
Your Answer: 75.0

The lottery has a 50/50 probability that it will either pay out 41 or 96
Your Answer: 68.505859375

The lottery has a 50/50 probability that it will either pay out 57 or 98
Your Answer: 77.490234375

The lottery has a 50/50 probability that it will either pay out 49 or 74
Your Answer: 61.4990234375

The lottery has a 50/50 probability that it will either pay out 50 or 61
Your Answer: 55.4931640625

The lottery has a 50/50 probability that it will either pay out 50 or 75
Your Answer: 62.5

The Piecewise Functions (intercept, slope, conditions):
---------------------------
0.0, 1.0x, if 0 ≤ x ≤ 25.0
0.0, 1.0x, if 25.0 < x ≤ 50.0
-0.06222222222221774, 1.0012444444444444x, if 50.0 < x ≤ 55.493164062